In [ ]:
# installing required packages for this notebook session
sc.install_pypi_package("boto3")

In [ ]:
# Total vehicles per day

import pyspark.sql.functions as F
import boto3

csv_out = "day,number_of_vehicles\n"

for day in range(1,32):
    traces = spark.read.parquet(f"s3://mobility-traces-sp/processed-data-avl-date/13-speed-calculation-filtered-columns/MO_1510{day}/")
    n_vehicles_day = traces.select(F.countDistinct("id_avl").alias("count")).collect()[0]["count"]
    
    csv_out += f"MO_1510{day},{n_vehicles_day}\n"


s3 = boto3.client('s3')

# writing results in S3
s3.put_object(Body=bytes(csv_out,"utf-8"), Bucket='mobility-traces-sp', Key='metrics-calculation/actives-buses/active-buses-per-day.csv')    


In [15]:
import pyspark.sql.functions as F

# Number of vehicles per hour per day
for day in range(1,32):
    traces = spark.read.parquet(f"s3://mobility-traces-sp/processed-data-avl-date/13-speed-calculation-filtered-columns/MO_1510{day}/")
    df = traces.groupby("hour_avl").agg(F.countDistinct("id_avl").alias("number_buses"))
    df.write.parquet(f"s3://mobility-traces-sp/metrics-calculation/actives-buses/active-buses-per-hour/MO_1510{day}/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
import pyspark.sql.functions as F

# Number of vehicles per hour and region per day
for day in range(1,32):
    traces = spark.read.parquet(f"s3://mobility-traces-sp/processed-data-avl-date/13-speed-calculation-filtered-columns/MO_1510{day}/")
    df = traces.groupby("hour_avl","region").agg(F.countDistinct("id_avl").alias("number_buses"))
    df.write.parquet(f"s3://mobility-traces-sp/metrics-calculation/actives-buses/active-buses-per-hour-per-region/MO_1510{day}/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…